# Multi-Modal RAG

## 학습 목표

**Multi-Modal RAG**: 텍스트와 이미지를 통합한 의미적 검색

## 학습 구성

**Part 1: Multi-Modal RAG** (1시간)
- Jina Embeddings v4 모델을 사용한 다국어 임베딩
- Qdrant Named Vectors를 활용한 Multi-Modal 저장
- Cross-Lingual Search 및 Late Fusion 전략

## 참고사항

- Multi-Modal 이미지는 `Day2/images/multimodal/` 경로에 있는 것을 사용합니다


In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

LAW_DATA_DIR = "./law"
PROCESSED_LAW_DIR = "./law/markdown"
print(f"  - 원본 법령 경로: {LAW_DATA_DIR}")
print(f"  - 변환 결과 경로: {PROCESSED_LAW_DIR}")

  - 원본 법령 경로: ./law
  - 변환 결과 경로: ./law/markdown


In [4]:
# ----------------------------------------------------------------------------
# OpenAI / OpenRouter 모델 초기화 헬퍼
# ----------------------------------------------------------------------------
from typing import Literal

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)


def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

---

## Part 1: Multi-Modal RAG

### 1.1 Multi-Modal RAG 개념 소개

**정의**: 텍스트와 이미지 등 여러 모달리티의 데이터를 통합하여 의미적 검색을 수행하는 RAG 시스템

**핵심 개념**:
- **Semantic Gap**: 서로 다른 모달리티 간의 의미적 차이를 임베딩 공간에서 해소
- **Shared Embedding Space**: 텍스트와 이미지를 동일한 벡터 공간에 매핑
- **Cross-Modal Search**: 텍스트로 이미지를 찾거나, 이미지로 텍스트를 찾는 검색

**활용 사례**:
1. 의료: "폐렴 X-ray 이미지"를 텍스트로 검색
2. 전자상거래: 이미지로 유사 상품 찾기
3. 문서 분석: PDF 스크린샷에서 관련 조항 검색
4. 법률: 차트/다이어그램으로 관련 조문 찾기

**참고**: [Qdrant Multilingual & Multimodal Search](https://qdrant.tech/documentation/multimodal-search/)


### 1.2 데이터셋 확인


In [ ]:
# ============================================================================
# Multi-Modal 데모 데이터셋 정의
# ============================================================================
from pathlib import Path
from typing import Any

import pandas as pd
from IPython.display import display

image_dir = DAY2_ROOT / "images" / "multimodal"

predefined_documents = [
    {
        "id": "mm_doc_001",
        "filename": "working_hours.png",
        "caption_ko": "근로기준법 제50조 근로시간 요약 다이어그램: 주 40시간, 연장 12시간 이내",
        "caption_en": "Diagram summarizing Korea Labor Standards Act Article 50: 40 hours/week, max 12 hours overtime",
        "metadata": {"topic": "근로시간", "law_reference": "근로기준법 제50조"},
    },
]

multimodal_documents: list[dict[str, Any]] = []
missing_assets: list[str] = []

for doc in predefined_documents:
    path = image_dir / doc["filename"]
    if path.exists():
        multimodal_documents.append(
            {
                "id": doc["id"],
                "caption_ko": doc["caption_ko"],
                "caption_en": doc["caption_en"],
                "image_path": path,
                "metadata": {**doc["metadata"], "filename": doc["filename"]},
            }
        )
    else:
        missing_assets.append(doc["filename"])

print(f"로딩된 멀티모달 샘플 수: {len(multimodal_documents)}")
if missing_assets:
    print(f"누락된 이미지 파일: {missing_assets}")

if not multimodal_documents:
    raise RuntimeError(
        "멀티모달 데이터셋이 비어 있습니다. Day2/images/multimodal 경로에 이미지를 추가하세요."
    )

# DataFrame 출력
display(
    pd.DataFrame(
        [
            {
                "id": doc["id"],
                "caption_ko": doc["caption_ko"][:60] + "...",
                "topic": doc["metadata"].get("topic", "N/A"),
            }
            for doc in multimodal_documents
        ]
    )
)

### 1.3 Multi-Modal 임베딩 모델

1. **Qwen3-VL-4B**


- 출처: [Qwen/Qwen3-VL-4B-Instruct](Qwen/Qwen3-VL-4B-Instruct)

2. **LiquidAI/LFM2-VL-3B**

- 출처: [https://huggingface.co/LiquidAI/LFM2-VL-3B](https://huggingface.co/LiquidAI/LFM2-VL-3B)

3. **MinerU2.5-2509-1.2B**

- 출처: [https://huggingface.co/opendatalab/MinerU2.5-2509-1.2B](https://huggingface.co/opendatalab/MinerU2.5-2509-1.2B)

4. **Jina Embeddings v4** (Qwen/Qwen2.5-VL-3B-Instruct 기반, Multi-Modal, 768차원)
   - Hugging Face Text Embeddings Inference (TEI)로 배포
   - 아쉽게도 `License 가 CC-BY-NC` 이라 프로덕션 환경에서 사용 불가
   - 출처: [https://huggingface.co/jinaai/jina-embeddings-v4](https://huggingface.co/jinaai/jina-embeddings-v4)

5. **Jina Reranker v3** (재정렬)
   - Late-Fusion 후 최종 재정렬, TEI로 배포 가능
   - 아쉽게도 `License 가 CC-BY-NC` 이라 프로덕션 환경에서 사용 불가
   - 출처: [https://huggingface.co/jinaai/jina-reranker-v3](https://huggingface.co/jinaai/jina-reranker-v3)


### 중국모델 이슈


In [ ]:
# ============================================================================
# Multi-Modal 임베딩 초기화 (Omni-Embed + Jina v4)
# ============================================================================
import sys

sys.path.insert(0, str(DAY2_ROOT / "src"))

import httpx
from mmrag.embeddings import HuggingfaceMMEmbedding

# 환경 변수 확인
EMBED_MM_URL = os.getenv("EMBED_MM_URL", "http://localhost:8001")
EMBED_TEXT_URL = os.getenv("EMBED_TEXT_URL", "http://localhost:8002")
RERANK_URL = os.getenv("RERANK_URL", "http://localhost:8003")

print(f"멀티모달 임베딩 서버: {EMBED_MM_URL}")
print(f"텍스트 임베딩 서버: {EMBED_TEXT_URL}")
print(f"리랭크 서버: {RERANK_URL}")

# Omni-Embed 클라이언트 (텍스트/이미지)
omni_embedder = HuggingfaceMMEmbedding(
    text_url=EMBED_MM_URL,
    image_url=EMBED_MM_URL,
    timeout=60.0,
    batch_size_text=16,
    batch_size_image=8,
)


# Jina Embeddings v4 클라이언트 (텍스트 전용, TEI 스타일)
class JinaTextEmbedding:
    def __init__(self, url: str):
        self.url = url.rstrip("/")
        self.client = httpx.Client(timeout=30.0)

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        resp = self.client.post(f"{self.url}/embed", json={"inputs": texts})
        resp.raise_for_status()
        return resp.json()

    def health_check(self) -> dict:
        try:
            resp = self.client.get(f"{self.url}/health")
            resp.raise_for_status()
            return resp.json()
        except Exception as e:
            return {"status": "error", "message": str(e)}


jina_embedder = JinaTextEmbedding(url=EMBED_TEXT_URL)

# 헬스체크
print("\n서버 헬스체크:")
print(f"  Omni-Embed: {omni_embedder.health_check()}")
print(f"  Jina v4: {jina_embedder.health_check()}")

NameError: name 'DAY2_ROOT' is not defined

#### 이미지 및 텍스트 임베딩 생성

Jina v4와 Omni-Embed를 사용하여 3종류의 임베딩을 생성합니다:
- Jina v4 텍스트 임베딩 (한국어/영어 평균)
- Omni-Embed 텍스트 임베딩 (한국어/영어 평균, 2048차원)
- Omni-Embed 이미지 임베딩 (2048차원)


In [ ]:
# ============================================================================
# Multi-Modal 임베딩 생성 (Omni-Embed + Jina v4)
# ============================================================================
import numpy as np

# 캡션 추출
captions_ko = [doc["caption_ko"] for doc in multimodal_documents]
captions_en = [doc["caption_en"] for doc in multimodal_documents]

# 1. Jina v4 텍스트 임베딩 (한국어/영어 평균)
print("🔄 Jina v4 텍스트 임베딩 생성 중...")
text_emb_ko = np.array(jina_embedder.embed_documents(captions_ko))
text_emb_en = np.array(jina_embedder.embed_documents(captions_en))
text_embeddings_jina = (text_emb_ko + text_emb_en) / 2

# 2. Omni-Embed 텍스트 임베딩 (한국어/영어 평균)
print("🔄 Omni-Embed 텍스트 임베딩 생성 중...")
text_emb_omni_ko = omni_embedder.embed_texts(captions_ko)
text_emb_omni_en = omni_embedder.embed_texts(captions_en)
text_embeddings_omni = (text_emb_omni_ko + text_emb_omni_en) / 2

# 3. Omni-Embed 이미지 임베딩
print("🔄 Omni-Embed 이미지 임베딩 생성 중...")
image_paths = [doc["image_path"] for doc in multimodal_documents]
image_embeddings = omni_embedder.embed_images(image_paths)

# 차원 확인
print("\n✅ 임베딩 생성 완료:")
print(f"   Jina 텍스트: {text_embeddings_jina.shape}")
print(f"   Omni 텍스트: {text_embeddings_omni.shape} (2048차원)")
print(f"   Omni 이미지: {image_embeddings.shape} (2048차원)")

# 차원 자동 감지
dim_jina = text_embeddings_jina.shape[1]
dim_omni = 2048

print("\n📏 자동 감지된 차원:")
print(f"   text (Jina): {dim_jina}")
print(f"   text_mm (Omni): {dim_omni}")
print(f"   image (Omni): {dim_omni}")

#### Qdrant Named Vectors 컬렉션 생성

3개의 Named Vectors로 멀티모달 임베딩을 저장합니다:
- `text`: Jina v4 텍스트 임베딩 (검색용)
- `text_mm`: Omni-Embed 텍스트 임베딩 (교차 검색용)
- `image`: Omni-Embed 이미지 임베딩


In [ ]:
# ============================================================================
# Qdrant Named Vectors 컬렉션 생성 및 업서트
# ============================================================================
import pandas as pd
from mmrag.qdrant_io import create_named_vectors_collection, upsert_multimodal_documents
from qdrant_client import QdrantClient, models

# Qdrant 클라이언트
qdrant_url = os.getenv("QDRANT_URL", "http://localhost:6333")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
print(f"📡 Qdrant endpoint: {qdrant_url}")

qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
MULTIMODAL_COLLECTION = "day2_multimodal_rag_v2"

# Named Vectors 컬렉션 생성 (차원 자동 감지)
created = create_named_vectors_collection(
    client=qdrant_client,
    collection_name=MULTIMODAL_COLLECTION,
    vector_configs={
        "text": dim_jina,
        "text_mm": dim_omni,
        "image": dim_omni,
    },
    distance=models.Distance.COSINE,
    on_disk=True,
)

if created:
    print(f"✅ 컬렉션 '{MULTIMODAL_COLLECTION}' 생성 완료")
else:
    print(f"⚠️ 컬렉션 '{MULTIMODAL_COLLECTION}' 이미 존재, 기존 사용")

# 문서 업서트
embeddings_dict = {
    "text": text_embeddings_jina,
    "text_mm": text_embeddings_omni,
    "image": image_embeddings,
}

num_uploaded = upsert_multimodal_documents(
    client=qdrant_client,
    collection_name=MULTIMODAL_COLLECTION,
    documents=multimodal_documents,
    embeddings=embeddings_dict,
)

print(f"\n✅ {num_uploaded}개 문서 업로드 완료")

# 업로드된 문서 확인
uploaded_df = pd.DataFrame(
    [
        {
            "id": doc["id"],
            "caption_ko": doc["caption_ko"][:50] + "...",
            "topic": doc["metadata"].get("topic", "N/A"),
        }
        for doc in multimodal_documents
    ]
)
display(uploaded_df)

#### 텍스트↔이미지 교차 검색

Omni-Embed는 텍스트와 이미지를 동일한 벡터 공간(2048차원)에 매핑하므로
교차 검색이 가능합니다:

1. **텍스트 → 이미지**: 텍스트 쿼리로 관련 이미지 검색
2. **이미지 → 텍스트**: 이미지 쿼리로 관련 텍스트(캡션) 검색
3. **Late-Fusion (RRF)**: 여러 검색 결과를 통합
4. **Reranker**: 최종 재정렬


In [ ]:
# ============================================================================
# 텍스트 ↔ 이미지 교차 검색 함수
# ============================================================================
from typing import Any

from IPython.display import display

from mmrag.qdrant_io import search_cross_modal, search_multimodal
from mmrag.ranking import late_fusion_with_rerank


def search_by_text(
    query: str,
    limit: int = 5,
    use_cross_modal: bool = False,
) -> pd.DataFrame:
    """텍스트로 검색 (텍스트 또는 이미지 결과)."""
    # 쿼리 임베딩 (Jina 또는 Omni)
    if use_cross_modal:
        # 교차 검색: 텍스트 → 이미지
        query_emb = omni_embedder.embed_texts([query])[0]
        results = search_cross_modal(
            client=qdrant_client,
            collection_name=MULTIMODAL_COLLECTION,
            query_vector=query_emb,
            query_vector_name="text_mm",
            target_vector_name="image",
            limit=limit,
        )
        search_type = "텍스트→이미지"
    else:
        # 일반 검색: 텍스트 → 텍스트
        query_emb = np.array(jina_embedder.embed_documents([query]))[0]
        results = search_multimodal(
            client=qdrant_client,
            collection_name=MULTIMODAL_COLLECTION,
            query_vector=query_emb,
            vector_name="text",
            limit=limit,
        )
        search_type = "텍스트→텍스트"

    # 결과 → DataFrame
    rows = []
    for hit in results:
        payload = hit.payload or {}
        rows.append(
            {
                "score": hit.score,
                "caption_ko": payload.get("caption_ko", "")[:50] + "...",
                "topic": payload.get("topic", "N/A"),
                "filename": payload.get("filename", "N/A"),
            }
        )

    df = pd.DataFrame(rows)
    print(f"🔍 {search_type} 검색: '{query}' → {len(df)}개 결과")
    return df


def search_by_image(
    image_path: str,
    limit: int = 5,
) -> pd.DataFrame:
    """이미지로 검색 (유사 이미지 또는 관련 텍스트)."""
    # 이미지 임베딩
    query_emb = omni_embedder.embed_images([image_path])[0]

    # 이미지 → 이미지 검색
    results = search_multimodal(
        client=qdrant_client,
        collection_name=MULTIMODAL_COLLECTION,
        query_vector=query_emb,
        vector_name="image",
        limit=limit,
    )

    # 결과 → DataFrame
    rows = []
    for hit in results:
        payload = hit.payload or {}
        rows.append(
            {
                "score": hit.score,
                "caption_ko": payload.get("caption_ko", "")[:50] + "...",
                "topic": payload.get("topic", "N/A"),
                "filename": payload.get("filename", "N/A"),
            }
        )

    df = pd.DataFrame(rows)
    print(f"🔍 이미지→이미지 검색 → {len(df)}개 결과")
    return df


def search_hybrid_with_rerank(
    query: str,
    top_k: int = 3,
) -> pd.DataFrame:
    """하이브리드 검색: 텍스트 + 교차 검색 → RRF → Reranker."""
    # 1. 텍스트→텍스트 검색
    query_emb_jina = np.array(jina_embedder.embed_documents([query]))[0]
    results_text = search_multimodal(
        client=qdrant_client,
        collection_name=MULTIMODAL_COLLECTION,
        query_vector=query_emb_jina,
        vector_name="text",
        limit=10,
    )

    # 2. 텍스트→이미지 교차 검색
    query_emb_omni = omni_embedder.embed_texts([query])[0]
    results_cross = search_cross_modal(
        client=qdrant_client,
        collection_name=MULTIMODAL_COLLECTION,
        query_vector=query_emb_omni,
        query_vector_name="text_mm",
        target_vector_name="image",
        limit=10,
    )

    # 3. 결과 → dict 형태로 변환
    def to_dict_list(results: list) -> list[dict[str, Any]]:
        return [
            {
                "id": hit.payload.get("doc_id", ""),
                "score": hit.score,
                "text": hit.payload.get("caption_ko", ""),
                **hit.payload,
            }
            for hit in results
        ]

    list_text = to_dict_list(results_text)
    list_cross = to_dict_list(results_cross)

    # 4. Late-Fusion (RRF) + Reranker
    fused = late_fusion_with_rerank(
        query=query,
        result_lists=[list_text, list_cross],
        rerank_url=RERANK_URL,
        rrf_k=60,
        fusion_top_k=20,
        final_top_n=top_k,
        text_key="caption_ko",
    )

    # 5. DataFrame 변환
    rows = []
    for doc in fused:
        rows.append(
            {
                "rerank_score": doc.get("rerank_score", 0.0),
                "rrf_score": doc.get("rrf_score", 0.0),
                "caption_ko": doc.get("caption_ko", "")[:60] + "...",
                "topic": doc.get("topic", "N/A"),
                "filename": doc.get("filename", "N/A"),
            }
        )

    df = pd.DataFrame(rows)
    print(f"🔍 하이브리드 검색 (RRF + Reranker): '{query}' → {len(df)}개 결과")
    return df


# 데모 실행
print("=" * 70)
print("데모 1: 텍스트→텍스트 검색")
display(search_by_text("근로시간 제한", limit=3))

print("\n" + "=" * 70)
print("데모 2: 텍스트→이미지 교차 검색")
display(search_by_text("주 40시간 근로", limit=3, use_cross_modal=True))

print("\n" + "=" * 70)
print("데모 3: 하이브리드 검색 (RRF + Reranker)")
display(search_hybrid_with_rerank("연차 휴가 일수", top_k=3))

### 1.4 Part 1 요약 및 참고자료

#### 주요 학습 내용

1. **Omni-Embed + Jina v4** 기반 3종류 임베딩 생성 및 Qdrant Named Vectors 저장
   - Jina v4 텍스트 임베딩 (검색용)
   - Omni-Embed 텍스트 임베딩 (교차 검색용, 2048차원)
   - Omni-Embed 이미지 임베딩 (2048차원)

2. **교차 검색 및 하이브리드 검색** 구현
   - 텍스트 → 텍스트 검색 (Jina v4)
   - 텍스트 → 이미지 교차 검색 (Omni-Embed)
   - 이미지 → 이미지 검색 (Omni-Embed)
   - RRF + Reranker 통합 파이프라인

3. **프로덕션 수준 기능**
   - Late-Fusion (Reciprocal Rank Fusion)
   - Jina Reranker v3를 통한 최종 재정렬
   - 배치 처리 및 재시도 로직

#### 참고자료
- [Qdrant Multimodal Search 가이드](https://qdrant.tech/documentation/multimodal-search/)
- [Omni-Embed Nemotron 3B](https://huggingface.co/nvidia/omni-embed-nemotron-3b)
- [Jina Embeddings v4](https://huggingface.co/jinaai/jina-embeddings-v4)
- [Jina Reranker v3](https://huggingface.co/jinaai/jina-reranker-v3)


### 1.5 RAG 생성 체인

검색된 컨텍스트를 LLM에 전달하여 최종 답변을 생성합니다.

**핵심 원칙**:
- 출처 표기 (파일명/캡션)
- 모르는 내용은 솔직하게 "모른다" 답변
- LangFuse 트레이싱으로 관측


In [ ]:
# ============================================================================
# RAG 체인 구현
# ============================================================================
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
)

# 프롬프트 템플릿
rag_template = """당신은 한국 법률/문서 전문가 AI입니다.
제공된 문서 컨텍스트를 기반으로 질문에 답변하세요.

**규칙**:
1. 출처를 명확히 표기하세요 (예: "근로기준법 제50조 다이어그램에 따르면...")
2. 컨텍스트에 없는 내용은 솔직하게 모른다고 답변하세요.
3. 법적 조언이 필요한 경우 전문가 상담을 권유하세요.

**문서 컨텍스트**:
{context}

**질문**: {question}

**답변**:"""

rag_prompt = PromptTemplate(
    template=rag_template,
    input_variables=["context", "question"],
)

# RAG 체인
rag_chain = rag_prompt | llm | StrOutputParser()


def multimodal_rag_answer(query: str, top_k: int = 3) -> str:
    """멀티모달 RAG 답변 생성."""
    # 1. 하이브리드 검색
    results_df = search_hybrid_with_rerank(query, top_k=top_k)

    # 2. 컨텍스트 구성
    context_parts = []
    for idx, row in results_df.iterrows():
        context_parts.append(
            f"[문서 {idx + 1}] {row['filename']}\n"
            f"내용: {row['caption_ko']}\n"
            f"주제: {row['topic']}\n"
            f"관련도: {row['rerank_score']:.3f}\n"
        )

    context = "\n".join(context_parts)

    # 3. LLM 답변 생성
    answer = rag_chain.invoke({"context": context, "question": query})

    return answer


# 데모 실행
print("=" * 70)
print("멀티모달 RAG 답변 생성 데모\n")

test_queries = [
    "근로시간은 어떻게 제한되나요?",
    "연차 휴가는 몇 일인가요?",
]

for q in test_queries:
    print(f"\n질문: {q}")
    print("-" * 70)
    answer = multimodal_rag_answer(q, top_k=3)
    print(f"답변:\n{answer}\n")
    print("=" * 70)

print("\nPart 1: Multi-Modal RAG 완료!")
print("   다음: Part 2 (GraphRAG)로 이동하세요.")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

**Multi-Modal RAG의 장점**:
1. 다양한 데이터 형식 통합 (텍스트, 이미지, 오디오)
2. 다국어 검색 용이
3. OCR 없이도 이미지 내용 검색 가능
4. 구현이 상대적으로 단순함

**Multi-Modal RAG의 단점**:
1. 문서 구조 정보 활용 어려움
2. 계층적 문맥 제공 불가
3. 대용량 모델 필요 (GPU 요구)
4. 이미지 품질에 민감


### 언제 Multi-Modal RAG를 선택할까요?

**적합한 시나리오**:
1. 의료 영상 검색 시스템
2. 전자상거래 상품 검색 (이미지 기반)
3. 교육 플랫폼 (다이어그램, 차트 검색)
4. 미디어 관리 시스템
5. 다국어 문서 검색

**선택 기준**:
- 텍스트와 이미지가 혼재된 데이터
- 문서 구조가 중요하지 않은 경우
- 빠른 구축이 필요한 경우
- 다양한 언어 지원이 필요한 경우

### Multi-Modal RAG 구현 시

- 이미지 품질 확인 (해상도, 선명도)
- 적절한 임베딩 모델 선택 (vdr-2b-multi-v1, CLIP 등)
- GPU 리소스 확보 (모델 크기 고려)
- 이미지 캡션 생성 전략 결정 (자동/수동)
- Named Vectors 구조 설계
- Late Fusion 전략 구현
- 다국어 지원 테스트
